In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

import numpy as np
import matplotlib.pyplot as plt

import spacy
#!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_lg

In [ ]:
import nltk
nltk.download('stopwords')

In [3]:
import sys
directory_path = "/Users/dragonzord/Documents/SchoolDocs/codeup-data-science/exercises/methodologies2/capstone/nlp-capstone/src"
sys.path.append(directory_path)
for p in sys.path:
    print(p)
    
# these imports must come after adding to path
import prepare_jag
from casenums import UniqueDataFrames

/Users/dragonzord/Documents/SchoolDocs/codeup-data-science/exercises/methodologies2/capstone/nlp-capstone/notebooks/scott
/Users/dragonzord/.vscode/extensions/ms-toolsai.jupyter-2022.2.1030672458/pythonFiles
/Users/dragonzord/.vscode/extensions/ms-toolsai.jupyter-2022.2.1030672458/pythonFiles/lib/python
/opt/miniconda3/envs/tf/lib/python37.zip
/opt/miniconda3/envs/tf/lib/python3.7
/opt/miniconda3/envs/tf/lib/python3.7/lib-dynload

/opt/miniconda3/envs/tf/lib/python3.7/site-packages
/opt/miniconda3/envs/tf/lib/python3.7/site-packages/IPython/extensions
/Users/dragonzord/.ipython
/Users/dragonzord/.local/lib/python3.7/site-packages
/Users/dragonzord/Documents/SchoolDocs/codeup-data-science/exercises/methodologies2/capstone/nlp-capstone/src


### don't run this, but it could be useful in the future

In [ ]:
# directory_path = "/Users/dragonzord/Documents/SchoolDocs/codeup-data-science/exercises/methodologies2/capstone/nlp-capstone/src/"
# for path in os.listdir(directory_path):
#     if path[-2:] == "py":
#         module_path = f"{directory_path}{path}"        
#         if module_path in sys.path: print(f"{module_path[-15:]} added successfully")
#         else: sys.path.append(module_path)

In [4]:
patient_notes = pd.read_csv("../../data/patient_notes.csv")
print(patient_notes.shape)
patient_notes.head(2)

(42146, 3)


,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student health clinic complaining of heart pounding. Mr. Cleveland's mother has given verbal consent for a history, physical examination, and treatment\r\n-began 2-3 months ago,sudden,intermittent for 2 days(lasting 3-4 min),worsening,non-allev/aggrav\r\n-associated with dispnea on exersion and rest,stressed out about school\r\n-reports fe feels like his heart is jumping out of his chest\r\n-ros:denies chest pain,dyaphoresis,wt loss,chills,fever,nausea,vomiting,pedal edeam\r\n-pmh:non,meds :aderol (from a friend),nkda\r\n-fh:father had MI recently,mother has thyroid dz\r\n-sh:non-smoker,mariguana 5-6 months ago,3 beers on the weekend, basketball at school\r\n-sh:no std"
1,1,0,"17 yo male with recurrent palpitations for the past 3 mo lasting about 3 - 4 min, it happened about 5 - 6 times since the beginning. One time durign a baskeball game two days ago light headedness, pressure in the chest, catching breath, but no fainting. During teh episodes no sweating. No diarrhea, no heat intolerance, no weight loss. Has tried aterol to be able to better concentrate, has received it from his roommate. ."


In [5]:
patient_notes[patient_notes["case_num"] == 0].head(2)

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student health clinic complaining of heart pounding. Mr. Cleveland's mother has given verbal consent for a history, physical examination, and treatment\r\n-began 2-3 months ago,sudden,intermittent for 2 days(lasting 3-4 min),worsening,non-allev/aggrav\r\n-associated with dispnea on exersion and rest,stressed out about school\r\n-reports fe feels like his heart is jumping out of his chest\r\n-ros:denies chest pain,dyaphoresis,wt loss,chills,fever,nausea,vomiting,pedal edeam\r\n-pmh:non,meds :aderol (from a friend),nkda\r\n-fh:father had MI recently,mother has thyroid dz\r\n-sh:non-smoker,mariguana 5-6 months ago,3 beers on the weekend, basketball at school\r\n-sh:no std"
1,1,0,"17 yo male with recurrent palpitations for the past 3 mo lasting about 3 - 4 min, it happened about 5 - 6 times since the beginning. One time durign a baskeball game two days ago light headedness, pressure in the chest, catching breath, but no fainting. During teh episodes no sweating. No diarrhea, no heat intolerance, no weight loss. Has tried aterol to be able to better concentrate, has received it from his roommate. ."


In [6]:
features = pd.read_csv("../../data/features.csv")
print(features.shape)
features.head(2)

(143, 3)


,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myocardial-infarction
1,1,0,Family-history-of-thyroid-disorder


In [7]:
features.count()

feature_num     143
case_num        143
feature_text    143
dtype: int64

In [8]:
features.feature_num.value_counts()

0      1
611    1
605    1
606    1
607    1
      ..
305    1
306    1
307    1
308    1
916    1
Name: feature_num, Length: 143, dtype: int64

In [9]:
features.case_num.value_counts()

5    18
8    18
2    17
9    17
3    16
0    13
1    13
6    12
4    10
7     9
Name: case_num, dtype: int64

In [10]:
features[(features["case_num"] == 0) & (features["feature_num"] == 1)]

,feature_num,case_num,feature_text
1,1,0,Family-history-of-thyroid-disorder


### Dictionary of case numbers and the phrases for which to look

In [ ]:
# case_num_feature_text = {}
# for i in range(10):
#     case_num_feature_text[i] = (features["feature_text"][features["case_num"] == i]).to_list()
# case_num_feature_text

In [ ]:
features.feature_text.value_counts().to_dict()

### the below creates too many observations

In [12]:
# notes_features = patient_notes.merge(right=features, left_on="case_num", right_on="case_num")#.drop("case_num_y", axis=1).rename({"case_num_x": "case_num"}, axis=1)
# print(notes_features.shape)
# notes_features.head()

In [15]:
# notes_features[["feature_num", "feature_text", "case_num"]][(notes_features["feature_num"] == 1)]

In [16]:
# notes_features.isnull().sum()

In [17]:
# notes_features[notes_features["case_num"] == 0].head()

In [18]:
# notes_features["feature_num"].value_counts()

In [19]:
# notes_features["case_num"].value_counts()

In [20]:
# len(notes_features["case_num"].value_counts().index)

In [ ]:
# feature_dict = {i: [(notes_features["feature_text"][notes_features["case_num"] == i]).values] for i in range(len(notes_features["case_num"].value_counts().index))}
# feature_dict

In [21]:
# notes_features["pn_history"][notes_features["case_num"] == 0][:10]

In [6]:
# case_num_zero = notes_features["pn_history"][notes_features["case_num"] == 0][:1500]
# whole_case_num_zero = "\n".join(case_num_zero.to_list())
# whole_case_num_zero[:1958]

### end too many observations

In [12]:
d = {}
for i in features["case_num"].unique():
    d[i] = {a: features["feature_text"][features["feature_num"] == a].to_string()[5:].strip() for a in features["feature_num"][features["case_num"] == i]}

In [13]:
df = pd.DataFrame(data=[d.keys(), d.values()]).T
df.columns = ["case_num", "features"]

In [14]:
notes_features2 = patient_notes.merge(right=df, left_on="case_num", right_on="case_num")

In [15]:
notes_features2["clean_pn_history"] = notes_features2["pn_history"].apply(prepare_jag.basic_clean3).apply(prepare_jag.remove_stopwords, exclude_words=["no"])

### Using spacy

In [17]:
nlp = spacy.load("en_core_web_lg")

In [18]:
case_num_zero = notes_features2["pn_history"][notes_features2["case_num"] == 0][:1000]
whole_case_num_zero = "\n".join(case_num_zero.to_list())
doc = nlp(whole_case_num_zero)

In [ ]:
plt.figure(figsize=(10, 8))
spacy.displacy.render(doc, style='ent', jupyter=True);

In [21]:
case_num_zero = patient_notes["pn_history"][patient_notes["case_num"] == 0][:1500]
whole_case_num_zero = "\n".join(case_num_zero.to_list())

In [22]:
clean_note = prepare_jag.basic_clean3(whole_case_num_zero)
clean_note = prepare_jag.remove_stopwords(clean_note, exclude_words=["no"])
len(clean_note)

968544

In [24]:
doc_clean = nlp(clean_note)
# print(len(doc_clean))
# doc_clean[:200]

In [ ]:
spacy.displacy.render(doc_clean, style='ent', jupyter=True);

### This might be the way to go to find locations

In [25]:
# for token in doc[:200]:
#     if token.pos_ not in ["NUM", "PUNCT", "SYM"] and token.ent_type_ not in ["DATE", "TIME"]:
#         print(token.text, token.ent_iob_, token.ent_type_, token.pos_)

In [ ]:
# for token in doc[:100].noun_chunks:
#     print(token)
#     print({tok.text: tok.pos_ for tok in token if tok.pos_ not in ["NUM", "PUNCT", "SYM", "DATE"]})

In [ ]:
# colors = {"Annotation" :"linear-gradient(to right, #2980b9, #6dd5fa, #ffffff);" } 
# options = {"colors": colors}
#spacy.displacy.render(doc, style='ent', options=options , manual=True, jupyter=True);

In [ ]:
# for ent in doc[:10000].ents:
#     print(f"text: {ent.text}, start_char: {ent.start_char}, end_char: {ent.end_char}, label_: {ent.label_}, start: {ent.start}, ent_id: {ent.ent_id_}, label: {ent.label}, vector_norm: {ent.vector_norm}")

In [ ]:
# for text, labels in data:
#         doc=[]
#         #print(text)
#         doc = nlp(text)
    
#         ents = []
#         labels["entities"]=sorted(labels["entities"])
#         for start, end, label in labels["entities"]:
#             print(start,end,label)
#             ents.append(doc.char_span(start, end, label))
        
#         print(ents)

### testing casenums.py

In [26]:
udf = UniqueDataFrames("../../data/patient_notes.csv", "../../data/features.csv")
notes_features = udf.merge_dfs()

In [27]:
unique_casenum_dict_df = udf.each_dataframe(notes_features)

In [28]:
type(unique_casenum_dict_df)

dict

In [29]:
unique_casenum_dict_df.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [30]:
type(unique_casenum_dict_df[0])

pandas.core.frame.DataFrame

In [31]:
unique_casenum_dict_df[0].head(2)

,pn_num,case_num,pn_history,features
0,0,0,"17-year-old male, has come to the student health clinic complaining of heart pounding. Mr. Cleveland's mother has given verbal consent for a history, physical examination, and treatment\r\n-began 2-3 months ago,sudden,intermittent for 2 days(lasting 3-4 min),worsening,non-allev/aggrav\r\n-associated with dispnea on exersion and rest,stressed out about school\r\n-reports fe feels like his heart is jumping out of his chest\r\n-ros:denies chest pain,dyaphoresis,wt loss,chills,fever,nausea,vomiting,pedal edeam\r\n-pmh:non,meds :aderol (from a friend),nkda\r\n-fh:father had MI recently,mother has thyroid dz\r\n-sh:non-smoker,mariguana 5-6 months ago,3 beers on the weekend, basketball at school\r\n-sh:no std","{0: 'Family-history-of-MI-OR-Family-history-of-myocardial-infarction', 1: 'Family-history-of-thyroid-disorder', 2: 'Chest-pressure', 3: 'Intermittent-symptoms', 4: 'Lightheaded', 5: 'No-hair-changes-OR-no-nail-changes-OR-no-temperature-intolerance', 6: 'Adderall-use', 7: 'Shortness-of-breath', 8: 'Caffeine-use', 9: 'heart-pounding-OR-heart-racing', 10: 'Few-months-duration', 11: '17-year', 12: 'Male'}"
1,1,0,"17 yo male with recurrent palpitations for the past 3 mo lasting about 3 - 4 min, it happened about 5 - 6 times since the beginning. One time durign a baskeball game two days ago light headedness, pressure in the chest, catching breath, but no fainting. During teh episodes no sweating. No diarrhea, no heat intolerance, no weight loss. Has tried aterol to be able to better concentrate, has received it from his roommate. .","{0: 'Family-history-of-MI-OR-Family-history-of-myocardial-infarction', 1: 'Family-history-of-thyroid-disorder', 2: 'Chest-pressure', 3: 'Intermittent-symptoms', 4: 'Lightheaded', 5: 'No-hair-changes-OR-no-nail-changes-OR-no-temperature-intolerance', 6: 'Adderall-use', 7: 'Shortness-of-breath', 8: 'Caffeine-use', 9: 'heart-pounding-OR-heart-racing', 10: 'Few-months-duration', 11: '17-year', 12: 'Male'}"


In [32]:
unique_casenum_dict_df[5].head(2)

,pn_num,case_num,pn_history,features
20192,50001,5,"26 yo F presents for follow up after ED visit two week ago where she had palpitations associated w/ numb fingers on both hands. Pt has a 5 yr history of untreated palpitation. The palpitations have no situational or environmental trigger. They are increasing in severity and freqency. She denies any weight change, skin change. The palpitations are accompanied by SOB, nausea, throat tightening, feeling hot, cold, and clammy and feeling of dread. \r\nROS: neg execpt as listed above\r\nPMH: none\r\nALL: NKDA\r\nMed: none\r\nSocHx: denies EtOH, tobacco, recrteational drugs, is sexually active w/ her boyfried and they always use condoms.","{500: 'Onset-5-years-ago', 501: 'Female', 502: 'No-caffeine-use', 503: 'Associated-SOB-OR-Associated-shortness-of-breath', 504: 'Episodes-of-heart-racing', 505: 'Recent-visit-to-emergency-department-with-negative-workup', 506: 'No-chest-pain', 507: 'No-illicit-drug-use', 508: 'Associated-nausea', 509: 'Increased-frequency-recently', 510: 'Associated-feeling-of-impending-doom', 511: 'Episodes-last-15-to-30-minutes', 512: 'Associated-throat-tightness', 513: 'Feels-hot-OR-Feels-clammy', 514: 'Episode-of-hand-numbness-OR-Episode-of-finger-numbness', 515: 'Fatigue-OR-Difficulty-concentrating', 516: 'Increased-stress', 517: '26-year'}"
20193,50002,5,"26 yo F c/o worsening heart palpitations for the past 3 weeks. Has had heart palpitations for the past 5 years, and nothing alleviates or exacerbates the symptoms. For the last 2 weeks the palpitations have occured every other day. Not associated with increased activity or emotional state of mind. Patient reports nausea, shortness of breath, and one incidence of cold/clammy fingertips. No vomitting, no bowel/bladder changes, and no muscle aches, joint pains, skin changes. \r\n\r\nROS: Negative except as above\r\nPMH: None\r\nPSH: None\r\nMeds: None\r\nAll: None\r\nSH: No alcohol, tobacco, illicit drug use\r\nFH: Non-contributory\r\n","{500: 'Onset-5-years-ago', 501: 'Female', 502: 'No-caffeine-use', 503: 'Associated-SOB-OR-Associated-shortness-of-breath', 504: 'Episodes-of-heart-racing', 505: 'Recent-visit-to-emergency-department-with-negative-workup', 506: 'No-chest-pain', 507: 'No-illicit-drug-use', 508: 'Associated-nausea', 509: 'Increased-frequency-recently', 510: 'Associated-feeling-of-impending-doom', 511: 'Episodes-last-15-to-30-minutes', 512: 'Associated-throat-tightness', 513: 'Feels-hot-OR-Feels-clammy', 514: 'Episode-of-hand-numbness-OR-Episode-of-finger-numbness', 515: 'Fatigue-OR-Difficulty-concentrating', 516: 'Increased-stress', 517: '26-year'}"
